In [4]:

import pandas as pd
import json
import chess.pgn
import io

# Open JSON file
with open('json_1.json', 'r') as file:
    data = json.load(file)

# Normalize the data
from pandas import json_normalize
normalized_df = json_normalize(data['games'])  # Adjust 'games' to your actual key

# Display the DataFrame

def extract_moves_from_pgn(pgn_string):
    pgn_io = io.StringIO(pgn_string)
    game = chess.pgn.read_game(pgn_io)
    board = game.board()
    moves = []
    for move in game.mainline_moves():
        moves.append(board.san(move))
        board.push(move)
    return moves

# Extract the moves
#moves = extract_moves_from_pgn(pgn_string)

def determine_opening(moves, openings):
    best_match = "Unknown Opening"
    max_matched_moves = 0

    for index, row in openings.iterrows():
        opening_name = row['name']
        opening_moves = row['opening_moves']
        matched_moves = 0

        # Count matched moves
        for i in range(min(len(moves), len(opening_moves))):
            if moves[i] == opening_moves[i]:
                matched_moves += 1
            else:
                break

        # Update best match if this opening matches more moves
        if matched_moves > max_matched_moves:
            best_match = opening_name
            max_matched_moves = matched_moves

    return best_match

# Read the TSV files into DataFrames
a_df = pd.read_csv('chess-openings/a.tsv', sep='\t')
b_df = pd.read_csv('chess-openings/b.tsv', sep='\t')
c_df = pd.read_csv('chess-openings/c.tsv', sep='\t')
d_df = pd.read_csv('chess-openings/d.tsv', sep='\t')
e_df = pd.read_csv('chess-openings/e.tsv', sep='\t')


# Combine the DataFrames
combined_df = pd.concat([a_df, b_df, c_df, d_df, e_df], ignore_index=True)

# Create a new column for the extracted moves
chess_moves = []

# Iterate over the rows of the combined DataFrame
for index, row in combined_df.iterrows():
    pgn_string = row['pgn']
    moves = extract_moves_from_pgn(pgn_string)
    chess_moves.append(moves)

# Add the new column to the DataFrame
combined_df['opening_moves'] = chess_moves



#opening_name = determine_opening(moves, combined_df)

#loop through normalized_df on column pgn run extract_moves_from_pgn and determine_opening 
#and append the opening_name to the normalized_df

#rewrite the loop to update the opening_name column in the normalized_df



for index, row in normalized_df.iterrows():
    pgn_string = row['pgn']
    moves = extract_moves_from_pgn(pgn_string)
    #why is extract_moves_from_pgn not defined?
    opening_name = determine_opening(moves, combined_df)
    normalized_df.at[index, 'opening_name'] = opening_name
    

In [5]:
normalized_df

,url,pgn,time_control,end_time,rated,tcn,uuid,initial_setup,fen,start_time,...,white.username,white.uuid,black.rating,black.result,black.@id,black.username,black.uuid,accuracies.white,accuracies.black,opening_name
0,https://www.chess.com/game/daily/644030175,"[Event ""Let's Play!""]\n[Site ""Chess.com""]\n[Da...",1/86400,1714610273,False,mCYIbs0SfAZJCJSJAJ!Tdm90mv8?ltTJsJ0Rgm5QcD98eh...,1cef1292-f681-11ee-8ea8-30e1f301000b,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,8/8/8/8/8/5K2/6Qk/8 b - - 16 71,1.712674e+09,...,aaronSmatta,2a0ff9aa-b37d-11ea-bdd5-756c1c59b54e,1162,checkmated,https://api.chess.com/pub/player/jyan_500,jyan_500,df2699c2-c63c-11e4-8000-000000000000,NaN,NaN,Sicilian Defense: Closed
1,https://www.chess.com/game/live/108792604287,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300+5,1715054634,True,mC0Kbs!TfAZRlt3VnDWOgvXHAr6EegYQcuOGiq90pxEZDK...,824090ab-0c24-11ef-aa9f-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2k2Q2/7R/2p5/4p3/6P1/3PB1KP/1pr5/8 b - -,NaN,...,jyan_500,df2699c2-c63c-11e4-8000-000000000000,895,checkmated,https://api.chess.com/pub/player/chesscheff,Chesscheff,14218644-b195-11df-8003-000000000000,83.64,76.12,"Bishop's Opening: Vienna Hybrid, Spielmann Attack"
2,https://www.chess.com/game/live/109054173603,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300+5,1715316136,False,mCYIbs0SlBIBdB5QBdSKnDZRgv6Efm!Teg90pxEvmv8!cu...,40a3a444-0e85-11ef-aa9f-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2Rr2k1/5p1p/3p2p1/1P2b3/P3P3/5B1P/2Q3P1/4q2K w...,NaN,...,jyan_500,df2699c2-c63c-11e4-8000-000000000000,1289,win,https://api.chess.com/pub/player/faisalsaifi0-...,faisalsaifi0-inactive,dd375f58-4528-11ec-8f62-7956eac321c1,NaN,NaN,Sicilian Defense: Marshall Gambit
3,https://www.chess.com/game/daily/652513337,"[Event ""Let's Play!""]\n[Site ""Chess.com""]\n[Da...",1/86400,1715373593,False,mCYIbs5QnDZRgv2Ufm92eh!Tlt8?cu6EpxEvmv2VoET8dl...,fd715b52-081c-11ef-a1b2-26023701000b,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,3r4/8/1p3N2/p3p1P1/4P3/QB1q4/P2n2K1/1k2R3 b - ...,1.714611e+09,...,jyan_500,df2699c2-c63c-11e4-8000-000000000000,1323,checkmated,https://api.chess.com/pub/player/aaronsmatta,aaronSmatta,2a0ff9aa-b37d-11ea-bdd5-756c1c59b54e,64.97,62.61,Sicilian Defense: Grand Prix Attack
4,https://www.chess.com/game/live/109143667273,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300+5,1715405134,False,mCYInDZJbs0Sgx!Tlt5QCJSJsH7GclGHkAJAtAHPdm90ec...,66dc43d3-0f56-11ef-829c-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,r3r1k1/pp3ppp/2n2b2/2p3P1/2P2P2/6QN/Pq5P/2KR1B...,NaN,...,KunjChandak,31af0e88-5423-11ec-b291-4dfcc4dd2261,1412,win,https://api.chess.com/pub/player/jyan_500,jyan_500,df2699c2-c63c-11e4-8000-000000000000,NaN,NaN,"Bird Opening: Dutch Variation, Batavo Gambit"
5,https://www.chess.com/game/live/109144207851,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300+5,1715405161,False,mC2UlB92,ee751828-0f56-11ef-829c-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,rnbqk1nr/ppppppbp/6p1/8/3PP3/8/PPP2PPP/RNBQKBN...,NaN,...,jyan_500,df2699c2-c63c-11e4-8000-000000000000,1444,resigned,https://api.chess.com/pub/player/kunjchandak,KunjChandak,31af0e88-5423-11ec-b291-4dfcc4dd2261,NaN,NaN,Modern Defense
6,https://www.chess.com/game/live/109144240675,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",600,1715405535,False,mCZJCJ7JbsJRgv!Tlt6Ecu5Qfm86eg3Npx0SdlQKuDKvmv...,660d3e90-0f57-11ef-829c-6cfe544c0428,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,2kr3r/ppp2pp1/3bpn2/7p/8/2NP1P1P/PPPQ1P2/R4RK1...,NaN,...,jyan_500,df2699c2-c63c-11e4-8000-000000000000,1337,resigned,https://api.chess.com/pub/player/chivesena,Chivesena,5ea29f60-ec45-11ee-9f45-37b6a443044b,NaN,NaN,Scandinavian Defense: Bronstein Variation
7,https://www.chess.com/game/live/109399257619,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",300+5,1715661423,True,mCYIbs0SfAWOiy5QnD!Tgv90eg8!de98ewZRltXPvMQBfn...,6510f5b6-11a9-11ef-829c-6c

In [6]:
# saved normalized_df to csv 
normalized_df.to_csv('normalized_df.csv', index=False)

In [13]:
# copy normalized_df to new_df with columns, url, pgn, white.rating, white.result, white.username, black.rating, black.result, black.username, opening_name, accuracies.white, accuracies.black
new_df = normalized_df[['url', 'pgn', 'white.username', 'white.rating', 'white.result', 'black.username', 'black.rating', 'black.result', 'opening_name', 'accuracies.white', 'accuracies.black']].copy()

In [14]:
new_df.to_csv('new_df.csv', index=False)

In [1]:

import pandas as pd
import json
import chess.pgn
import io

# Open JSON file
with open('stanley_json.json', 'r') as file:
    data = json.load(file)

# Normalize the data
from pandas import json_normalize
normalized_df = json_normalize(data['games'])  # Adjust 'games' to your actual key

# Display the DataFrame

def extract_moves_from_pgn(pgn_string):
    pgn_io = io.StringIO(pgn_string)
    game = chess.pgn.read_game(pgn_io)
    board = game.board()
    moves = []
    for move in game.mainline_moves():
        moves.append(board.san(move))
        board.push(move)
    return moves

# Extract the moves
#moves = extract_moves_from_pgn(pgn_string)

def determine_opening(moves, openings):
    best_match = "Unknown Opening"
    max_matched_moves = 0

    for index, row in openings.iterrows():
        opening_name = row['name']
        opening_moves = row['opening_moves']
        matched_moves = 0

        # Count matched moves
        for i in range(min(len(moves), len(opening_moves))):
            if moves[i] == opening_moves[i]:
                matched_moves += 1
            else:
                break

        # Update best match if this opening matches more moves
        if matched_moves > max_matched_moves:
            best_match = opening_name
            max_matched_moves = matched_moves

    return best_match

# Read the TSV files into DataFrames
a_df = pd.read_csv('chess-openings/a.tsv', sep='\t')
b_df = pd.read_csv('chess-openings/b.tsv', sep='\t')
c_df = pd.read_csv('chess-openings/c.tsv', sep='\t')
d_df = pd.read_csv('chess-openings/d.tsv', sep='\t')
e_df = pd.read_csv('chess-openings/e.tsv', sep='\t')


# Combine the DataFrames
combined_df = pd.concat([a_df, b_df, c_df, d_df, e_df], ignore_index=True)

# Create a new column for the extracted moves
chess_moves = []

# Iterate over the rows of the combined DataFrame
for index, row in combined_df.iterrows():
    pgn_string = row['pgn']
    moves = extract_moves_from_pgn(pgn_string)
    chess_moves.append(moves)

# Add the new column to the DataFrame
combined_df['opening_moves'] = chess_moves



#opening_name = determine_opening(moves, combined_df)

#loop through normalized_df on column pgn run extract_moves_from_pgn and determine_opening 
#and append the opening_name to the normalized_df

#rewrite the loop to update the opening_name column in the normalized_df



for index, row in normalized_df.iterrows():
    pgn_string = row['pgn']
    moves = extract_moves_from_pgn(pgn_string)
    #why is extract_moves_from_pgn not defined?
    opening_name = determine_opening(moves, combined_df)
    normalized_df.at[index, 'opening_name'] = opening_name
    

In [2]:
# copy normalized_df to new_df with columns, url, pgn, white.rating, white.result, white.username, black.rating, black.result, black.username, opening_name, accuracies.white, accuracies.black
new_df = normalized_df[['url', 'pgn', 'white.username', 'white.rating', 'white.result', 'black.username', 'black.rating', 'black.result', 'opening_name', 'accuracies.white', 'accuracies.black']].copy()

In [5]:
new_df

,url,pgn,white.username,white.rating,white.result,black.username,black.rating,black.result,opening_name,accuracies.white,accuracies.black
0,https://www.chess.com/game/live/108419431673,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",Voirien3,833,win,TTTstanley,852,checkmated,"English Opening: Anglo-Indian Defense, Queen's...",75.39,66.36
1,https://www.chess.com/game/live/108487267709,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",TTTstanley,843,resigned,Juxkun,820,win,"Benoni Defense: Benoni-Indian Defense, Kingsid...",NaN,NaN
2,https://www.chess.com/game/live/108761431511,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",chessboi74,886,win,TTTstanley,836,resigned,Benoni Defense: Franco-Sicilian Hybrid,80.87,73.38
3,https://www.chess.com/game/live/107242047806,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",Twix7778,572,win,TTTstanley,534,resigned,Zukertort Opening: Queen's Gambit Invitation,NaN,NaN
4,https://www.chess.com/game/live/107243041108,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",TTTstanley,574,win,david25252,519,abandoned,Benoni Defense: Franco-Sicilian Hybrid,NaN,NaN
5,https://www.chess.com/game/live/107246642366,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",FadianChess,549,resigned,TTTstanley,609,win,Benoni Defense: Franco-Sicilian Hybrid,NaN,NaN
6,https://www.chess.com/game/live/107246741196,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",TTTstanley,844,win,Faroosmgs,819,resigned,Queen's Pawn Game,NaN,NaN
7,https://www.chess.com/game/live/107300637468,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",Andrii7777777777,836,checkmated,TTTstanley,852,win,King's Indian Attack: Sicilian Variation,NaN,NaN
8,https://www.chess.com/game/live/109546859387,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",TTTstanley,844,resigned,hobbatron32,863,win,Dutch Defense,68.92,89.17
9,https://www.chess.com/game/live/107329053608,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",LagunaBrian,834,abandoned,TTTstanley,852,win,"French Defense: Advance Variation, Steinitz Va...",84.78,88.56


In [6]:
new_df.to_csv('new_df1.csv', index=False)

In [4]:
!pwd

/Users/stanleytan/Downloads
